In [37]:
# snap_manager.py —— 约 60 行的最小可运行 Undo / Redo
from collections import deque
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Copy
from OCC.Core.BRepTools import breptools_Write, breptools_Read
from OCC.Core.TopoDS import TopoDS_Shape
import tempfile, os, zlib

# ---------- 工具函数 ----------
def deepcopy_shape(shape: TopoDS_Shape) -> TopoDS_Shape:
    """内存内 1:1 深拷贝形状"""
    return BRepBuilderAPI_Copy(shape).Shape()  # 实际 API 详见 pythonocc demos:contentReference[oaicite:4]{index=4}

def serialize_shape(shape: TopoDS_Shape) -> bytes:
    """BREP → gzip 字节串；用于大模型省内存或持久化"""
    fd, path = tempfile.mkstemp(suffix=".brep"); os.close(fd)
    breptools_Write(shape, path)               # 官方示例同款:contentReference[oaicite:5]{index=5}
    data = zlib.compress(open(path, "rb").read())
    os.remove(path); return data

def deserialize_shape(data: bytes) -> TopoDS_Shape:
    fd, path = tempfile.mkstemp(suffix=".brep"); os.close(fd)
    open(path, "wb").write(zlib.decompress(data))
    shape = TopoDS_Shape(); breptools_Read(shape, path)
    os.remove(path); return shape

# ---------- 核心类 ----------
class SnapManager:
    def __init__(self, init_shape, copy_fn=deepcopy_shape, restore_fn=lambda x: x):
        self._copy, self._restore = copy_fn, restore_fn
        self._undo, self._redo = deque([self._copy(init_shape)]), deque()

    @property
    def shape(self):                  # 取当前形状
        return self._restore(self._undo[-1])

    def snapshot(self, shape):        # 每次完成一条命令后调用
        self._undo.append(self._copy(shape)); self._redo.clear()

    def can_undo(self): return len(self._undo) > 1
    def can_redo(self): return bool(self._redo)

    def undo(self):                   # ⇧Z
        if self.can_undo(): self._redo.append(self._undo.pop())
        return self.shape

    def redo(self):                   # ⇧Y
        if self.can_redo(): self._undo.append(self._redo.pop())
        return self.shape


In [38]:
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox
from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer

# 初始形状（10×20×30 盒）
shape0 = BRepPrimAPI_MakeBox(10, 20, 30).Shape()
snap = SnapManager(shape0)          # 管理器实例

renderer = JupyterRenderer()  
renderer.DisplayShape(shape0)
renderer.Display()


In [39]:
from OCC.Core.gp import gp_Trsf, gp_Vec
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform

def translate(dx, dy, dz):
    trsf = gp_Trsf(); trsf.SetTranslation(gp_Vec(dx, dy, dz))
    new_shape = BRepBuilderAPI_Transform(snap.shape, trsf, True).Shape()
    snap.snapshot(new_shape)        # 记录历史
    renderer.EraseAll()                      # 更新显示
    renderer.DisplayShape(new_shape,shape_color = "#111111")
    renderer.Display()

translate(20, 0, 0)   # 第 1 步
translate(0, 10, 0)  # 第 2 步


In [36]:


renderer.EraseAll() 
renderer.DisplayShape(snap.undo(), shape_color="#009999")
renderer.Display()






In [23]:

renderer.EraseAll() 
renderer.DisplayShape(snap.redo(), shape_color="#990099")
renderer.Display()